In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm_notebook
from tqdm import tqdm
from itertools import product
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
import requests
import pandas as pd
from requests.auth import HTTPBasicAuth

endpoint = 'https://www.daloopa.com'
TICKER = 'SIVB'
response = requests.get(f"{endpoint}/api/v1/export/{TICKER}",auth=HTTPBasicAuth('jedgore1@gmail.com','MgcAvxZBUpXYbi5t0RIpFfnruGYrv4kDTvPdxK7_w7DpozIvQpQ8EQ'),stream=True)

df = pd.read_csv(response.raw)


In [3]:
df['category'].unique()

array(['Guidance', 'KPIs', 'Segmental breakdown', 'Income statement',
       'Balance sheet', 'Cash flow statement', 'GAAP to Non- GAAP',
       'Premises and equipment', 'Other breakdown'], dtype=object)

In [4]:
df = df.loc[df['category']=='Balance sheet']
df = df.loc[df['span']=='quarterly']

In [5]:
assets_list = ['Cash and cash equivalents',
       'Available-for-sale securities, at fair value',
       'Held-to-maturity securities, at amortized cost and, net of allowance for credit losses\xa0',
       'Non-marketable and other equity securities',
       'Investment securities', 'Loans amortized cost',
       'Allowance for credit losses loans', 'Net loans', 'Goodwill',
       'Premises and equipment, net of accumulated depreciation and amortization',
       'Other intangible assets, net', 'Lease right-of-use assets',
       'Accrued interest receivable and other assets', 'Total assets']
liabilities_list = ['Non-Interest-bearing demand deposits',
       'Interest-bearing deposits', 'Total deposits',
       'Short-term borrowings', 'Lease liabilities', 'Other liabilities',
       'Long-term debt', 'Total liabilities']
equity_list = ['Preferred stock',
       'Common stock ', 'Additional paid in capital', 'Retained earnings',
       'Accumulated other comprehensive loss',
       'Total SVBFG stockholders equity', 'Non-controlling interests',
       'Total equity', 'Total liabilities and total equity',
       'Cash and due from banks',
       'Interest bearing deposits with the Federal Reserve Bank',
       'Interest bearing deposits with other institutions',
       'Securities purchased under agreements to resell',
       'Other short-term investment securities']

In [6]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
assets = []
for asset_type in assets_list:
    temp_df = df.loc[df['label']==asset_type]
    temp_df['value_raw']=temp_df['value_raw']
    assets.append(temp_df)

liabilities = []
for liability_type in liabilities_list:
    temp_df = df.loc[df['label']==liability_type]
    temp_df['value_raw']=temp_df['value_raw']*(-1)
    liabilities.append(temp_df)

df_assets = pd.concat(assets).reset_index()
df_liabilities = pd.concat(liabilities).reset_index()

#df_assets = df_assets.pivot(index='label',columns = 'filing_date',values='value_raw').reset_index()
#df_liabilities = df_liabilities.pivot(index='label',columns = 'filing_date',values='value_raw').reset_index()

#df_plot = pd.concat([df_assets,df_liabilities])


In [7]:
df_assets.head()

,index,id,ticker,company_name,label,category,span,calendar_period,fiscal_period,fiscal_date,...,value_normalized,source_link,series_id,filing_date,series_id_relations,series_tag,restated,title,capiq_ticker,is_transition_period
0,5068,44686049,SIVB,Svb Financial Group,Cash and cash equivalents,Balance sheet,quarterly,2013Q1,2013Q1,2013-03-31,...,1519.249,https://www.daloopa.com/src/44686049,3351229,2013-04-25,NaN,NaN,False,Balance sheet | Assets: | Investment securitie...,NasdaqGS:SIVB,False
1,5069,44686431,SIVB,Svb Financial Group,Cash and cash equivalents,Balance sheet,quarterly,2013Q2,2013Q2,2013-06-30,...,873.251,https://www.daloopa.com/src/44686431,3351229,2013-07-25,NaN,NaN,False,Balance sheet | Assets: | Investment securitie...,NasdaqGS:SIVB,False
2,5070,44686641,SIVB,Svb Financial Group,Cash and cash equivalents,Balance sheet,quarterly,2013Q3,2013Q3,2013-09-30,...,1942.744,https://www.daloopa.com/src/44686641,3351229,2013-10-24,NaN,NaN,False,Balance sheet | Assets: | Investment securitie...,NasdaqGS:SIVB,False
3,5071,44688448,SIVB,Svb Financial Group,Cash and cash equivalents,Balance sheet,quarterly,2013Q4,2013Q4,2013-12-31,...,1538.779,https://www.daloopa.com/src/44688448,3351229,2014-01-23,NaN,NaN,False,Balance sheet | Assets: | Investment securitie...,NasdaqGS:SIVB,False
4,5072,44688368,SIVB,Svb Financial Group,Cash and cash equivalents,Balance sheet,quarterly,2014Q1,2014Q1,2014-03-31,...,3862.464,https://www.daloopa.com/src/44688368,3351229,2014-04-24,NaN,NaN,False,Balance sheet | Assets: | Investment securitie...,NasdaqGS:SIVB,False


In [8]:
df_assets.pivot(index='label',columns = 'filing_date',values='value_raw').reset_index().head()

filing_date,label,2013-04-25,2013-07-25,2013-10-24,2014-01-23,2014-04-24,2014-07-24,2014-10-23,2015-01-22,2015-04-23,...,2020-10-22,2021-01-21,2021-04-22,2021-07-22,2021-10-21,2022-01-20,2022-04-21,2022-07-02,2022-10-20,2023-01-19
0,Accrued interest receivable and other assets,354.402,413.639,397.432,465.110,458.531,465.792,506.964,559.231,762.971,...,2316.979,3205.825,4184.114,2650.000,2074.000,1928.000,3088.000,2777.000,3523.000,3082.000
1,Allowance for credit losses loans,-112.205,-119.571,-124.734,-142.886,-123.542,-120.728,-129.061,-165.359,-167.875,...,-512.958,-447.765,-391.751,-396.000,-398.000,-422.000,-421.000,-545.000,-557.000,-636.000
2,"Available-for-sale securities, at fair value",10908.163,10043.341,10209.917,11986.821,12843.099,11672.790,13333.436,13540.655,13746.923,...,25904.324,30912.438,25986.471,23876.000,22984.000,27221.000,25991.000,26223.000,26711.000,26069.000
3,Cash and cash equivalents,1519.249,873.251,1942.744,1538.779,3862.464,2649.831,1872.537,1796.062,1308.003,...,15687.776,17674.763,21254.859,23959.000,18940.000,14619.000,20606.000,15398.000,13968.000,13803.000
4,Goodwill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,137.823,142.685,142.685,143.000,344.000,375.000,375.000,375.000,375.000,375.000


In [9]:
df_liabilities.pivot(index='label',columns = 'filing_date',values='value_raw').reset_index().head()

filing_date,label,2013-04-25,2013-07-25,2013-10-24,2014-01-23,2014-04-24,2014-07-24,2014-10-23,2015-01-22,2015-04-23,...,2020-10-22,2021-01-21,2021-04-22,2021-07-22,2021-10-21,2022-01-20,2022-04-21,2022-07-02,2022-10-20,2023-01-19
0,Interest-bearing deposits,-5271.321,-5476.516,-5891.263,-6578.619,-7162.075,-8116.998,-8662.067,-9759.817,-8134.989,...,-27264.791,-35462.567,-39710.109,-44579.000,-55794.000,-63352.000,-70137.000,-73976.000,-82831.000,-92356.000
1,Lease liabilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-246.652,-259.554,-287.413,-277.000,-390.000,-388.000,-374.000,-377.000,-429.000,-413.000
2,Long-term debt,-457.194,-455.938,-455.744,-455.216,-454.770,-454.462,-453.764,-453.443,-802.917,...,-843.430,-843.628,-1338.183,-1834.000,-1925.000,-2570.000,-2571.000,-3367.000,-3368.000,-5370.000
3,Non-Interest-bearing demand deposits,-14038.587,-13213.558,-14105.728,-15894.360,-18314.830,-20235.549,-22461.068,-24583.682,-25716.586,...,-57508.229,-66519.240,-84439.997,-101259.000,-115388.000,-125851.000,-127997.000,-113969.000,-93988.000,-80753.000
4,Other liabilities,-359.380,-330.394,-358.905,-404.586,-407.573,-559.073,-517.462,-483.493,-686.501,...,-3067.221,-3971.974,-6411.705,-3449.000,-2733.000,-2587.000,-2817.000,-2721.000,-2889.000,-3041.000


In [10]:
df_assets_plot = df_assets.pivot(index = 'filing_date',columns = 'label',values = 'value_raw').reset_index()

In [11]:
df_assets_plot.head()

label,filing_date,Accrued interest receivable and other assets,Allowance for credit losses loans,"Available-for-sale securities, at fair value",Cash and cash equivalents,Goodwill,"Held-to-maturity securities, at amortized cost and, net of allowance for credit losses",Investment securities,Lease right-of-use assets,Loans amortized cost,Net loans,Non-marketable and other equity securities,"Other intangible assets, net","Premises and equipment, net of accumulated depreciation and amortization",Total assets
0,2013-04-25,354.402,-112.205,10908.163,1519.249,NaN,NaN,12123.951,NaN,8844.890,8732685.000,1215.788,NaN,65.713,22796.000
1,2013-07-25,413.639,-119.571,10043.341,873.251,NaN,NaN,11298766.000,NaN,9622.172,9502601.000,1255.425,NaN,65.644,22153901.000
2,2013-10-24,397.432,-124.734,10209.917,1942.744,NaN,NaN,11635055.000,NaN,9824.982,9700248.000,1425.138,NaN,65.385,23740.864
3,2014-01-23,465.110,-142.886,11986.821,1538.779,NaN,NaN,13582315.000,NaN,10906.386,10763500.000,1595.494,NaN,67.485,26417.189
4,2014-04-24,458.531,-123.542,12843.099,3862.464,NaN,NaN,14613555.000,NaN,10833.908,10710366.000,1770.456,NaN,66.123,29711039.000


In [12]:
assets_list
#df_assets[assets_list]

['Cash and cash equivalents',
 'Available-for-sale securities, at fair value',
 'Held-to-maturity securities, at amortized cost and, net of allowance for credit losses\xa0',
 'Non-marketable and other equity securities',
 'Investment securities',
 'Loans amortized cost',
 'Allowance for credit losses loans',
 'Net loans',
 'Goodwill',
 'Premises and equipment, net of accumulated depreciation and amortization',
 'Other intangible assets, net',
 'Lease right-of-use assets',
 'Accrued interest receivable and other assets',
 'Total assets']

In [13]:
import plotly.graph_objects as go
import numpy
  
  
# creating random data through randomint 
# function of numpy.random 
np.random.seed(42)
  
random_x= np.random.randint(1,101,100) 
random_y= np.random.randint(1,101,100)
  
x = ['A', 'B', 'C', 'D']
  
plot = go.Figure()
  
plot.add_trace(go.Scatter(
    name = 'Data 1',
    x = x,
    y = [100, 200, 500, 673],
    stackgroup='one'
   ))
  
plot.add_trace(go.Scatter(
    name = 'Data 2',
    x = x,
    y = [56, 123, 982, 213],
    stackgroup='one'
   )
)
  
plot.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
!pip install nbformat

  Using cached nbformat-5.7.3-py3-none-any.whl (78 kB)
  Using cached fastjsonschema-2.16.3-py3-none-any.whl (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 12.9 MB/s eta 0:00:00


In [ ]:
1/0
res = pd.read_html('https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=117283648&type=HTML&symbol=SIVB&companyName=SVB+Financial+Group&formType=10-K&formDescription=Annual+report+pursuant+to+Section+13+or+15%28d%29&dateFiled=2023-02-24&CK=719739')


In [ ]:
df = res[14]

In [ ]:
df

In [ ]:
# df2 = df.loc[:, (np.arange(len(df.columns)) + 2) % 3 != 0]
# r = df2.iloc[:, 0:1 ]
# a = df2.loc[:, (np.arange(len(df2.columns))) % 2 != 0]
# a = a.iloc[:, 0:3 ]
# a
# keep_cols = []
# for col in df.columns:
#     if 'Yield'  in str(col):
#         keep_cols.append(col)
# keep_cols
# df2 = df2[keep_cols]
# y = df2.loc[:, (np.arange(len(df2.columns)) + 1) % 2 != 0]
# y = y.iloc[:, 0:3 ]
# y
df2 = df.replace('NaN', pd.NA).dropna(thresh=15,axis=1)
df2 = df2.loc[:, (np.arange(len(df2.columns))) % 2 != 0]
df2.columns = ['row_header','avg_balance_2022','ie_2022','yield_rate_2022','avg_balance_2021','ie_2021','yield_rate_2021','avg_balance_2020','ie_2020','yield_rate_2020',]
cols = [c for c in df2.columns if c.lower()[:2] != 'ie']
df_yr_1=df2[cols]
#df2
df_yr_1
#df_yr_1 = pd.concat([r,a,y],axis=1)
#df_yr_1.columns = ['row_header','avg_balance_2022','avg_balance_2021','avg_balance_2020','yield_rate_2022','yield_rate_2021','yield_rate_2020']
#df_yr_1

In [ ]:
res = pd.read_html('https://app.quotemedia.com/data/downloadFiling?webmasterId=90423&ref=115681262&type=HTML&symbol=SIVB&companyName=SVB+Financial+Group&formType=10-K&formDescription=Annual+report+pursuant+to+Section+13+or+15%28d%29&dateFiled=2021-03-01&CK=719739')

In [ ]:
df = res[14]
df2 = df.replace('NaN', pd.NA).dropna(thresh=15,axis=1)
df2 = df2.loc[:, (np.arange(len(df2.columns))) % 2 != 0]
df2.columns = ['row_header','avg_balance_2020','ie_2020','yield_rate_2020','avg_balance_2019','ie_2019','yield_rate_2019','avg_balance_2018','ie_2018','yield_rate_2018',]

cols = [c for c in df2.columns if (c.lower()[:1] == 'y' or c.lower()[:1]=='r') or c.lower()[:1]=='a']
df_yr_2=df2[cols]
cols = [c for c in df_yr_2.columns if (c.lower()[-4:] != '2020' or c.lower()[:1]=='r')]
df_yr_2=df_yr_2[cols]
df_yr_2

df_yr_2 = df_yr_2.loc[~df_yr_2['row_header'].isna()]


In [ ]:
df_yr_2

In [ ]:
df_merge = df_yr_1.merge(df_yr_2,how='left',on='row_header')
df_merge = df_merge[~df_merge['row_header'].isna()]
#cols = [c for c in df_merge.columns if c.lower()[:2] != 'av']
df_merge = df_merge[cols]

In [ ]:
df_merge

In [ ]:
pd.merge(df_yr1_merge,df_yr2_merge)


In [ ]:
df2 = df.loc[:, (np.arange(len(df.columns)) + 2) % 3 != 0]
df2
#r = df2.iloc[:, 0:1 ]
#r
a = df2.loc[:, (np.arange(len(df2.columns))) % 3 != 0]
a = a.iloc[:, 0:3 ]
a
df2.rename(columns=df2.iloc[4],inplace=True)
df2
keep_cols = []
for col in df2.columns:
    if 'Yield'  in str(col):
        keep_cols.append(col)
keep_cols
df2 = df2[keep_cols]
df2

#y = df2.loc[:, (np.arange(len(df2.columns)) + 1) % 2 != 0]
#y = y.iloc[:, 0:3 ]
#y

# df_yr_2 = pd.concat([r,a,y],axis=1)
#df_yr_2.columns = ['row_header','avg_balance_2022','avg_balance_2021','avg_balance_2020','yield_rate_2022','yield_rate_2021','yield_rate_2020']
# df_yr_2

In [ ]:
df = res[14]
df.columns = list(df.loc[3:3].values)
keep_cols = []
for col in df.columns:
    if 'nan' not in str(col):
        keep_cols.append(col)
keep_cols
df = df[keep_cols]
df = df.iloc[:, 8: ]
df